In [1]:
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import re

from __future__ import division

/Users/barricklab/anaconda/lib/python2.7/site-packages/matplotlib/font_manager.py:273: UserWarning: Matplotlib is building the font cache using fc-list. This may take a moment.
  warnings.warn('Matplotlib is building the font cache using fc-list. This may take a moment.')


### Import peptide table from file

In [21]:
all_pept = pd.DataFrame.from_csv("data/all_pept.tsv",sep="\t")
print all_pept.head()
modified_pepts = all_pept[all_pept.MassShift != 0]
print modified_pepts.head()
unmodified_pepts = all_pept[all_pept.MassShift == 0]
print modified_pepts.head()

  DataSet  Index  ObservedMW  Charge  CalculatedMW  DeltaMass  Score  \
0    100a    128    986.5384       2      986.5396    -0.0013     23   
1    100a    164    986.5396       2      986.5396    -0.0000     21   
2    100a    167    986.5377       2      986.5396    -0.0019     22   
3    100a    173    987.5440       2      987.5396     0.0043     19   
4    100a    190    986.5389       2      986.5396    -0.0008     26   

   Probability          Peptide       giID       ...        COGs  \
0       0.3035    K.LAGDLETLR.S  254160296       ...           -   
1       0.1744    K.LAGDLETLR.S  254160296       ...           -   
2       0.1571    K.LAGDLETLR.S  254160296       ...           -   
3       0.2249  K.L+1AGDLETLR.S  254160296       ...           -   
4       0.8113    K.LAGDLETLR.S  254160296       ...           -   

                                   ProteinName ProteinProduct_y Time  BiolRep  \
0  outer membrane protein assembly factor YaeT     YP_003043404    6        3

### Group peptides by
- Timepoint (i.e. count of all peptides at each timepoint)
- Timepoint x Mass Shift (count all peptides containing individual modifications for each time point)
- Timepoint x Mass Shift x Biological Replicate

In [5]:
pept_total_by_timepoint = all_pept.groupby(['Time']).count().Index
mod_spectrum_by_timepoint = all_pept.groupby(['Time','MassShift']).count().Index
mod_spectrum_by_timepoint_biorep = all_pept.groupby(['MassShift','Time','BiolRep']).count().Index

In [6]:
all_time_sum_biorep = mod_spectrum_by_timepoint_biorep.unstack().sum(level='MassShift')
all_time_sum_pooled = mod_spectrum_by_timepoint_biorep.fillna(0).sum(level='MassShift')
all_time_sum_pooled = all_time_sum_pooled.reindex(index=np.arange(-200,200),fill_value=0)
all_time_sum_pooled[0] = 0

### Use MODa output "Probability" column to calculate per-peptide FDR

Treat `1 - Probability` as the probability of type 1 error

Order peptides by probability, add peptides to the "passed" set until the sum of 1-P values / total passed is greater 
than the FDR cutoff

In [9]:
def calc_MODa_FDR(pept_df,fdr_cut,start_idx):
    """
    Filter pept_df (Dataframe of MODa output + protein info from parse_MODa.ipynb) to fdr_cut FDR
    
    start_idx allows the loop to jump to the specified index & calculate sums using (fast) pandas methods;
        mostly for debugging
    """
    # Sort peptides by probability
    pept_df_psort = pept_df.sort(columns=['Probability',],ascending=False,inplace=False)
    cut_idx = 0
    
    # Iterate over rows, adding each row's Probability to p_total
    for (idx,(df_ix,r)) in enumerate(pept_df_psort.iloc[start_idx:].iterrows()):
        idx += 1
        idx += start_idx
        p_total = idx - pept_df_psort.loc[:df_ix,'Probability'].sum()
        calc_fdr = p_total/idx
        if idx % 10000 == 0:
            print idx, p_total, calc_fdr
            
        # stop iteration when we reach the FDR cutoff
        if calc_fdr >= fdr_cut:
            cut_idx = idx
            break
    
    # Make a truncated copy of the original sorted data
    all_pept_fdr = pept_df_psort.iloc[:cut_idx].copy()
    
    # Add columns for modification location relative to the start of the PROTEIN 
    all_pept_fdr.loc[slice(None),'ModPosn_ProtCoord'] = (all_pept_fdr['PeptideStart'] + all_pept_fdr['ModifiedPosition']) - 2
    print all_pept_fdr.loc[(all_pept_fdr['PeptideStart'] == 1),slice(None)].head()
    all_pept_fdr.loc[(all_pept_fdr['PeptideStart'] == 1),'ModPosn_ProtCoord'] += 1
    all_pept_fdr.loc[((all_pept_fdr['PeptideStart'] == 1) & (all_pept_fdr['ModifiedPosition'] == -1)),'ModPosn_ProtCoord'] += 2
    print all_pept_fdr.loc[(all_pept_fdr['PeptideStart'] == 1),slice(None)].head()
    
    # Select modified and unmodified sets
    modified_pept_fdr = all_pept_fdr[all_pept_fdr['MassShift'] != 0]
    unmodified_pept_fdr = all_pept_fdr[all_pept_fdr['MassShift'] == 0]
    modified_pept_fdr_errorsum = modified_pept_fdr.shape[0] - modified_pept_fdr['Probability'].sum()
    unmodified_pept_fdr_errorsum = unmodified_pept_fdr.shape[0] - unmodified_pept_fdr['Probability'].sum()

    print "Total peptides: %d" % all_pept_fdr.shape[0]
    print "Modified Peptides: %d Error_sum: %f FDR: %f" % (modified_pept_fdr.shape[0],modified_pept_fdr_errorsum,(modified_pept_fdr_errorsum / modified_pept_fdr.shape[0]))
    print "Unodified Peptides: %d Error_sum: %f FDR: %f"  % (unmodified_pept_fdr.shape[0],unmodified_pept_fdr_errorsum,(unmodified_pept_fdr_errorsum / unmodified_pept_fdr.shape[0]))
    return (all_pept_fdr,modified_pept_fdr,unmodified_pept_fdr)

In [10]:
all_pept_fdr5, modified_pept_fdr5, unmodified_pept_fdr5 = calc_MODa_FDR(all_pept,.05,1970000)

/Users/barricklab/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py:9: FutureWarning: sort(columns=....) is deprecated, use sort_values(by=.....)


1980000 98744.2441042 0.0498708303556
        DataSet  Index  ObservedMW  Charge  CalculatedMW  DeltaMass  Score  \
1477411     25a  42489   1846.9254       2     1846.9276    -0.0022     97   
1475163     25a  24537   1876.8806       2     1876.8771     0.0035     91   
1476491     25a  48213   1936.8988       3     1936.9146    -0.0158     83   
856213      18b  65908   2296.1638       3     2296.1718    -0.0080     78   
856219      18b  66103   2274.1732       3     2274.1718     0.0014     72   

         Probability                       Peptide       giID  \
1477411          1.0         -.MNVVILDTGCANLNSVK.S  254162011   
1475163          1.0         -.MIQEQTMLNVADNSGAR.R  254163238   
1476491          1.0        -.M-2DQTYSLESFLNHVQK.R  254161819   
856213           1.0  -.MFENITAA+22PADPILGLADLFR.A  254161042   
856219           1.0     -.MFENITAAPADPILGLADLFR.A  254161042   

               ...                                               ProteinName  \
1477411        ...    

In [11]:
all_pept_fdr1, modified_pept_fdr1, unmodified_pept_fdr1 = calc_MODa_FDR(all_pept,.01,1470000)

/Users/barricklab/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py:9: FutureWarning: sort(columns=....) is deprecated, use sort_values(by=.....)


        DataSet  Index  ObservedMW  Charge  CalculatedMW  DeltaMass  Score  \
1477411     25a  42489   1846.9254       2     1846.9276    -0.0022     97   
1475163     25a  24537   1876.8806       2     1876.8771     0.0035     91   
1476491     25a  48213   1936.8988       3     1936.9146    -0.0158     83   
856213      18b  65908   2296.1638       3     2296.1718    -0.0080     78   
856219      18b  66103   2274.1732       3     2274.1718     0.0014     72   

         Probability                       Peptide       giID  \
1477411          1.0         -.MNVVILDTGCANLNSVK.S  254162011   
1475163          1.0         -.MIQEQTMLNVADNSGAR.R  254163238   
1476491          1.0        -.M-2DQTYSLESFLNHVQK.R  254161819   
856213           1.0  -.MFENITAA+22PADPILGLADLFR.A  254161042   
856219           1.0     -.MFENITAAPADPILGLADLFR.A  254161042   

               ...                                               ProteinName  \
1477411        ...         imidazole glycerol phosphate synt

##  Get counts for unmodified positions (protein coordinates)

### Stack Peptides on full-length protein, and sum total unmodified for each position in the protein

In [12]:
def generate_unmodified_positions_table(unmod_pepts,mod_pepts):
    """
    Get cumulative counts of unmodified positions (i.e. both from unmodified peptides and non-modified positions
    of modified peptides)
    
    Build a table of counts across time points separating biological replicates
    """
    
    # get maximum protein length so we can format the table
    max_prot_len = max([unmod_pepts['Length'].max(),mod_pepts['Length'].max()]) + 1
    
    # group peptides by parent protein
    unmod_pepts_grp = unmod_pepts.groupby(['LocusTag','Locus','BiolRep','Time'])
    mod_pepts_grp = mod_pepts.groupby(['LocusTag','Locus','BiolRep','Time'])
    
    # build a combined index from the modified and unmodified sets
    all_grp_idx = pd.concat([unmod_pepts_grp.count()['Index'],mod_pepts_grp.count()['Index']],axis=1).index
    
    # New data frames for aggregate modifications; each row represents a protein, number of
    # columns equal to length of the longest protein in the database
    # 
    # After these are filled with totals for each position, we'll stack on sequence index and drop 
    # empty rows, so we end up with one row per position for each protein
    unmod_pept_posn_df = pd.DataFrame(index=all_grp_idx, columns=np.arange(1,max_prot_len)).fillna(0)
    
    # Separate table for the AA at each position; will merge to the position count table when we're done
    unmod_pept_posn_AAs_df = pd.DataFrame(index=all_grp_idx, columns=np.arange(1,max_prot_len))
    
    # Group (i.e. protein) counts, for bookkeeping
    ngroups = len(unmod_pepts_grp.groups)
    mod_ngroups = len(mod_pepts_grp.groups)
    
    for (n,(ix,grp)) in enumerate(unmod_pepts_grp):
        # Sum across all protein positions for each group
        unmod_pept_range_sum(unmod_pept_posn_df.loc[ix,slice(None)],grp)# = grp.apply(pept_range_df,axis=1,args=(unmod_pept_posn_df.columns,)).sum()
        
        # Get AA for each position
        pept_AAs(unmod_pept_posn_AAs_df.loc[ix,slice(None)],grp)
        if n % 100 == 0:
            print (ix,unmod_pept_posn_df.loc[ix,slice(None)].sum())
            print (ix,unmod_pept_posn_AAs_df.loc[ix,slice(None)].dropna())
            print "(%d of %d, %.2f %% of unmod)" % (n,ngroups,(n/ngroups) * 100)
    
    for (n,(ix,grp)) in enumerate(mod_pepts_grp):
        # Same as above for the modified set, but skipping any modified residues in each peptide
        mod_pept_range_sum(unmod_pept_posn_df.loc[ix,slice(None)],grp)# = grp.apply(pept_range_df,axis=1,args=(unmod_pept_posn_df.columns,)).sum()
        pept_AAs(unmod_pept_posn_AAs_df.loc[ix,slice(None)],grp)
        if n % 100 == 0:
            print (ix,unmod_pept_posn_df.loc[ix,slice(None)].sum())
            print (ix,unmod_pept_posn_AAs_df.loc[ix,slice(None)].dropna())
            print "(%d of %d, %.2f %% of mod)" % (n,mod_ngroups,(n/mod_ngroups) * 100)

    # Stack rows and drop empty positions        
    unmod_pept_posn_df.columns = unmod_pept_posn_df.columns.set_names(['ModPosn_ProtCoord'])
    unmod_pept_posn_AAs_df.columns = unmod_pept_posn_AAs_df.columns.set_names(['ModPosn_ProtCoord'])
    unmod_pept_posn_AAs_df_stck = unmod_pept_posn_AAs_df.stack()
    unmod_pept_posn_df_stck = unmod_pept_posn_df[unmod_pept_posn_df > 0].stack()
    
    # Move around indexes to match other tables
    unmod_pept_posn_AAs_df_stck = unmod_pept_posn_AAs_df_stck.unstack('BiolRep').fillna(method='ffill',axis=1).fillna(method='bfill',axis=1).stack('BiolRep')
    unmod_pept_posn_AAs_df_nix = unmod_pept_posn_AAs_df_stck.reset_index().set_index(['LocusTag','Locus','ModPosn_ProtCoord','BiolRep','Time'])
    unmod_pept_posncnt_df_nix = unmod_pept_posn_df_stck.reset_index().set_index(['LocusTag','Locus','ModPosn_ProtCoord','BiolRep','Time'])
    
    # Join AA table and count table
    unmod_pept_posn_df = pd.concat([unmod_pept_posn_AAs_df_nix,unmod_pept_posncnt_df_nix],axis=1)
    unmod_pept_posn_df.columns = pd.Index(['ModifiedResidue','unmodCount'])
    
    # Fix indexes
    unmod_pept_posn_df = unmod_pept_posn_df.reset_index().set_index(['LocusTag','Locus','ModPosn_ProtCoord','ModifiedResidue','BiolRep','Time'])
    unmod_pept_posn_df = unmod_pept_posn_df.unstack('BiolRep').unstack('Time').fillna(0)
    
    return unmod_pept_posn_df.sort_index()

### Miscellaneous utility functions

"""
Think these are obsolete...

def fixed_len_arange(st,end,alen):
    r = np.arange(st,end)
    a = np.zeros(alen)
    a[:r.shape[0]] = r
    return a

def pept_range_df(x,cols):
    ser = pd.Series.from_array([1,] * ((x['PeptideEnd']+1) - x['PeptideStart']),index=range(x['PeptideStart'],x['PeptideEnd']+1))
    ser = ser.reindex(index=cols)
    return ser
"""
def unmod_pept_range_sum(pept_line,group):
    for (ix,row) in group.iterrows():
        pept_line[row['PeptideStart'] - 1:row['PeptideEnd']] += 1
    
def mod_pept_range_sum(pept_line,group):
    for (ix,row) in group.iterrows():
        pept_line.loc[row['PeptideStart']:row['ModPosn_ProtCoord'] - 1] += 1
        pept_line.loc[row['ModPosn_ProtCoord'] + 1:row['PeptideEnd']] += 1
        
def pept_AAs(pept_line,group):
    grp_unique_pepts = pd.concat([group['PeptideStart'],group['PeptideEnd'],group['PeptideSeq']],axis=1)
    for (ix,row) in grp_unique_pepts.iterrows():
        pept_line[row['PeptideStart'] - 1:row['PeptideEnd']] = np.array(list(re.search('\.(\S+)\.',row['PeptideSeq']).group(1)))


In [13]:
# Generate table of per-position counts of unmodified peptides
# This can take a while to run

unmodified_posn_fdr1 = generate_unmodified_positions_table(unmodified_pept_fdr1,modified_pept_fdr1)
print unmodified_posn_fdr1

(('ECB_00002', 'thrA', 1, 3), 2940)
(('ECB_00002', 'thrA', 1, 3), 43     I
44     T
45     N
46     H
47     L
48     V
49     A
50     M
51     I
52     E
53     K
54     T
55     I
56     S
57     G
58     Q
59     D
60     A
61     L
62     P
63     N
64     I
65     S
66     D
67     A
68     E
69     R
70     I
71     F
72     A
      ..
783    H
784    Y
785    Y
786    Q
787    P
788    L
789    P
790    L
791    V
792    L
793    R
794    G
795    Y
796    G
797    A
798    G
799    N
800    D
801    V
802    T
803    A
804    A
805    G
806    V
807    F
808    A
809    D
810    L
811    L
812    R
Name: (ECB_00002, thrA, 1, 3), dtype: object)
(0 of 39578, 0.00 % of unmod)
(('ECB_00006', 'yaaA', 3, 4), 200)
(('ECB_00006', 'yaaA', 3, 4), 64     F
65     H
66     D
67     W
68     Q
69     P
70     D
71     F
72     T
73     P
74     E
75     N
76     A
77     R
85     G
86     D
87     V
88     Y
89     T
90     G
91     L
92     Q
93     A
94     E
95     T
96     F
97     S
9

### Distribution of modifications over proteins (count & distribution)

In [16]:
def get_protein_pept_counts(pept_subset_df,pept_all_df):
    subset_proteins = pept_subset_df.groupby(['LocusTag','Locus']).count().Index
    subset_protein_total_pepts = pept_all_df[pept_all_df['LocusTag'].isin(subset_proteins.reset_index()['LocusTag'])]
    subset_protein_total_cnt = subset_protein_total_pepts.groupby(['LocusTag','Locus']).count().Index
    subset_protein_cnts = pd.DataFrame(data={'ModifiedPeptides':subset_proteins,
                                           'UnmodifiedPeptides':subset_protein_total_cnt - subset_proteins,
                                           'TotalPeptides':subset_protein_total_cnt})

    total_proteins = pept_all_df.groupby(['LocusTag','Locus']).count().Index
    nonsubset_protein_pepts = pept_all_df[~pept_all_df['LocusTag'].isin(subset_proteins.reset_index()['LocusTag'])]
    nonsubset_proteins = nonsubset_protein_pepts.groupby(['LocusTag','Locus']).count().Index
    return (subset_protein_cnts,nonsubset_proteins,total_proteins)

def get_protein_x_time_pept_counts(pept_subset_df,pept_all_df):
    subset_proteins = pept_subset_df.groupby(['LocusTag','Locus','Time','BiolRep']).count().Index
    subset_protein_total_pepts = pept_all_df[pept_all_df['LocusTag'].isin(subset_proteins.reset_index()['LocusTag'])]
    subset_protein_total_cnt = subset_protein_total_pepts.groupby(['LocusTag','Locus','Time','BiolRep']).count().Index
    subset_protein_cnts = pd.DataFrame(data={'ModifiedPeptides':subset_proteins,
                                           'UnmodifiedPeptides':subset_protein_total_cnt - subset_proteins,
                                           'TotalPeptides':subset_protein_total_cnt})

    total_proteins = pept_all_df.groupby(['LocusTag','Locus','Time','BiolRep']).count().Index
    nonsubset_protein_pepts = pept_all_df[~pept_all_df['LocusTag'].isin(subset_proteins.reset_index()['LocusTag'])]
    nonsubset_proteins = nonsubset_protein_pepts.groupby(['LocusTag','Locus','Time','BiolRep']).count().Index
    
    subset_proteins = subset_proteins.unstack()
    subset_proteins = subset_proteins.unstack()
    subset_proteins = subset_proteins.fillna(0)

    subset_protein_cnts = subset_protein_cnts.unstack()
    subset_protein_cnts = subset_protein_cnts.unstack()
    subset_protein_cnts = subset_protein_cnts.fillna(0)

    nonsubset_proteins = nonsubset_proteins.unstack()
    nonsubset_proteins = nonsubset_proteins.unstack()
    nonsubset_proteins = nonsubset_proteins.fillna(0)

    total_proteins = total_proteins.unstack()
    total_proteins = total_proteins.unstack()
    total_proteins = total_proteins.fillna(0)
    return (subset_proteins,nonsubset_proteins,total_proteins,subset_protein_cnts)

def get_uniqpept_x_time_pept_counts(pept_subset,pept_all_df):
    subset_pept_cnts = pept_subset.groupby(['PeptideSeq','Time','BiolRep']).count().Index
    subset_total_pepts = pept_all_df[pept_all_df['PeptideSeq'].isin(subset_pept_cnts.reset_index()['PeptideSeq'])]
    subset_total_cnts = subset_total_pepts.groupby(['PeptideSeq','Time','BiolRep']).count().Index
    subset_cnts = pd.DataFrame(data={'ModifiedPeptides':subset_pept_cnts,
                                           'UnmodifiedPeptides':subset_total_cnts - subset_pept_cnts,
                                           'TotalPeptides':subset_total_cnts})
    nonsubset_pepts = pept_all_df[~pept_all_df['PeptideSeq'].isin(subset_pept_cnts.reset_index()['PeptideSeq'])]
    nonsubset_cnts = nonsubset_pepts.groupby(['PeptideSeq','Time','BiolRep']).count().Index
    total_cnts = pept_all_df.groupby(['PeptideSeq','Time','BiolRep']).count().Index
    
    subset_pept_cnts = subset_pept_cnts.unstack()
    subset_pept_cnts = subset_pept_cnts.unstack()
    subset_pept_cnts = subset_pept_cnts.fillna(0)

    subset_cnts = subset_cnts.unstack()
    subset_cnts = subset_cnts.unstack()
    subset_cnts = subset_cnts.fillna(0)

    nonsubset_cnts = nonsubset_cnts.unstack()
    nonsubset_cnts = nonsubset_cnts.unstack()
    nonsubset_cnts = nonsubset_cnts.fillna(0)

    total_cnts = total_cnts.unstack()
    total_cnts = total_cnts.unstack()
    total_cnts = total_cnts.fillna(0)

    return (subset_pept_cnts,nonsubset_cnts,total_cnts,subset_cnts)

def get_count_info(pept_df,uniqpept_df,prot_df):
    pept_total = pept_df.shape[0]
    errorsum = pept_df.shape[0] - pept_df['Probability'].sum()
    info = [pept_total,
            errorsum,
            (errorsum / pept_total),
            uniqpept_df.shape[0],
            prot_df.shape[0]]
    return info


In [19]:
# FDR Calc summary table
fdr_summ_midx = pd.MultiIndex.from_product([('Unfiltered',"1% FDR","5% FDR"),(" ","Modified","Unmodified")])
fdr_summ_df = pd.DataFrame(index=fdr_summ_midx,columns=['Total PSSMs','Error Sum','False Discovery Rate','Unique Peptides','Proteins'])

all_pept_modified_uniqpept, all_pept_unmodified_uniqpept, all_pept_total_uniqpept, all_pept_mod_unmod_uniqpept_cnts = get_uniqpept_x_time_pept_counts(modified_pepts,all_pept)
all_pept_fdr1_modified_uniqpept, all_pept_fdr1_unmodified_uniqpept, all_pept_fdr1_total_uniqpept, all_pept_fdr1_mod_unmod_uniqpept_cnts = get_uniqpept_x_time_pept_counts(modified_pept_fdr1,all_pept_fdr1)
all_pept_fdr5_modified_uniqpept, all_pept_fdr5_unmodified_uniqpept, all_pept_fdr5_total_uniqpept, all_pept_fdr5_mod_unmod_uniqpept_cnts = get_uniqpept_x_time_pept_counts(modified_pept_fdr5,all_pept_fdr5)

all_pept_modified_prot, all_pept_unmodified_prot, all_pept_total_prot, all_pept_mod_unmod_protein_cnts = get_protein_x_time_pept_counts(modified_pepts,all_pept)
all_pept_fdr1_modified_prot, all_pept_fdr1_unmodified_prot, all_pept_fdr1_total_prot, all_pept_fdr1_mod_unmod_protein_cnts = get_protein_x_time_pept_counts(modified_pept_fdr1,all_pept_fdr1)
all_pept_fdr5_modified_prot, all_pept_fdr5_unmodified_prot, all_pept_fdr5_total_prot, all_pept_fdr5_mod_unmod_protein_cnts = get_protein_x_time_pept_counts(modified_pept_fdr5,all_pept_fdr5)

In [22]:
# Format the table into something readable
fdr_summ_df.loc[('Unfiltered',' '),:] = get_count_info(all_pept,all_pept_total_uniqpept,all_pept_total_prot)
fdr_summ_df.loc[('Unfiltered','Modified'),:] = get_count_info(modified_pepts,all_pept_modified_uniqpept,all_pept_modified_prot)
fdr_summ_df.loc[('Unfiltered','Unmodified'),:] = get_count_info(unmodified_pepts,all_pept_unmodified_uniqpept,all_pept_unmodified_prot)

fdr_summ_df.loc[('1% FDR',' '),:] = get_count_info(all_pept_fdr1,all_pept_fdr1_total_uniqpept,all_pept_fdr1_total_prot)
fdr_summ_df.loc[('1% FDR','Modified'),:] = get_count_info(modified_pept_fdr1,all_pept_fdr1_modified_uniqpept,all_pept_fdr1_modified_prot)
fdr_summ_df.loc[('1% FDR','Unmodified'),:] = get_count_info(unmodified_pept_fdr1,all_pept_fdr1_unmodified_uniqpept,all_pept_fdr1_unmodified_prot)

fdr_summ_df.loc[('5% FDR',' '),:] = get_count_info(all_pept_fdr5,all_pept_fdr5_total_uniqpept,all_pept_fdr5_total_prot)
fdr_summ_df.loc[('5% FDR','Modified'),:] = get_count_info(modified_pept_fdr5,all_pept_fdr5_modified_uniqpept,all_pept_fdr5_modified_prot)
fdr_summ_df.loc[('5% FDR','Unmodified'),:] = get_count_info(unmodified_pept_fdr5,all_pept_fdr5_unmodified_uniqpept,all_pept_fdr5_unmodified_prot)

print fdr_summ_df.to_latex(float_format=(lambda x: "%.3f" % x))

\begin{tabular}{lllllll}
\toprule
       &   & Total PSSMs &  Error Sum & False Discovery Rate & Unique Peptides & Proteins \\
\midrule
Unfiltered &   &     2527135 & 444225.241 &                0.176 &           32755 &     3544 \\
       & Modified &      606815 & 190762.077 &                0.314 &           25360 &     3478 \\
       & Unmodified &     1920320 & 253463.165 &                0.132 &            7395 &       66 \\
1\% FDR &   &     1473636 &  14736.377 &                0.010 &           19265 &     2121 \\
       & Modified &      197176 &   3214.449 &                0.016 &            8357 &     1690 \\
       & Unmodified &     1276460 &  11521.927 &                0.009 &           10908 &      431 \\
5\% FDR &   &     1980884 &  99044.212 &                0.050 &           22776 &     2445 \\
       & Modified &      360917 &  31863.077 &                0.088 &           13291 &     2188 \\
       & Unmodified &     1619967 &  67181.135 &                0.041 &    

### Output data tables

In [23]:
all_pept_fdr1.to_pickle("data/all_pept_fdr1.pck")
modified_pept_fdr1.to_pickle("data/modified_pept_fdr1.pck")
unmodified_pept_fdr1.to_pickle("data/unmodified_pept_fdr1.pck")
unmodified_posn_fdr1.to_pickle("data/unmodified_posn_fdr1.pck")

all_pept_fdr1.to_csv("data/all_pept_fdr1.tsv",sep="\t")
modified_pept_fdr1.to_csv("data/modified_pept_fdr1.tsv",sep="\t")
unmodified_pept_fdr1.to_csv("data/unmodified_pept_fdr1.tsv",sep="\t")
unmodified_posn_fdr1.to_csv("data/unmodified_posn_fdr1.tsv",sep="\t")
